In [ ]:
import sys
!{sys.executable} -m pip install ccxt

In [ ]:
path='./crypt.config'

import json
import ccxt

exchanges={
    'binance': ccxt.binance(),
    'gdax': ccxt.gdax()
}

configs=json.load(open(path))
myExchanges=configs['exchanges']
for myExchange in myExchanges:
    exchange_name=myExchange['exchange']
    exchange=exchanges[exchange_name]
    if 'apiKey' in myExchange:
        exchanges[exchange_name].apiKey=myExchange['apiKey']
    if 'secret' in myExchange:
        exchanges[exchange_name].secret=myExchange['secret']
    if 'password' in myExchange:
        exchanges[exchange_name].password=myExchange['password']

    exchanges[exchange_name].load_markets()
    for market in myExchange['markets']:
        print('exchange: %s,\tname: %s,\tpair: %s,\tval: %s' % (exchange_name, market['name'], market['pair'], exchange.fetch_ticker(market['pair'])['ask']))

balances=[]

binance_balances_all = exchanges['binance'].fetch_balance()
binance_balances = list(filter(lambda x: (float(x['free']) != 0 or float(x['locked']) != 0), binance_balances_all['info']['balances']))
BTC_USDT = exchanges['binance'].fetch_ticker('BTC/USDT')['bid']
balances.extend([(lambda x: {'currency': balance['asset'],
                             'quantity': float(x['free']) + float(x['locked']), 
                             'value': (float(x['free']) + float(x['locked'])) * BTC_USDT * (1 if (balance['asset'] == 'BTC') else exchanges['binance'].fetch_ticker('{}/BTC'.format(balance['asset']))['bid']),
                            })
                 (balance) for balance in binance_balances])

gdax_balances_all = exchanges['gdax'].fetch_balance()
gdax_balances = list(filter(lambda x: (float(x['balance']) != 0), gdax_balances_all['info']))
balances.extend([(lambda x: {'currency': balance['currency'],
                             'quantity': float(x['balance']), 
                             'value': float(x['balance']) if (balance['currency'] == 'USD') else 
                                      float(x['balance']) * exchanges['gdax'].fetch_ticker('{}/USD'.format(balance['currency']))['bid']})(balance) for balance in gdax_balances])

In [ ]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt

balancesDf = pd.DataFrame(balances).set_index('currency').groupby('currency').sum()
print(balancesDf)
print('\nTOTAL VALUE: ${}'.format(sum(map(lambda x: x['value'], balances))))

plt.pie(balancesDf['value'].tolist(), labels=balancesDf.index.values, autopct='%1.0f%%', pctdistance=1.1, labeldistance=1.2)
plt.show()

In [ ]:
#gdax order history

gdaxClosedOrders = exchanges['gdax'].fetchClosedOrders()
gdaxClosedOrdersDf = pd.DataFrame(gdaxClosedOrders)[['datetime', 'symbol', 'side', 'price', 'amount', 'cost']].sort_values(by=['symbol', 'datetime']).set_index('symbol')
print(gdaxClosedOrdersDf)

In [ ]:
#binance order history

binanceClosedOrders = []
for balance in binance_balances:
    market = balance['asset']
    if market != 'BTC':
        binanceClosedOrders.extend(exchanges['binance'].fetchMyTrades('{}/BTC'.format(market)))
binanceClosedOrdersDf = pd.DataFrame(binanceClosedOrders)[['datetime', 'symbol', 'side', 'price', 'amount', 'cost']].sort_values(by=['symbol', 'datetime'])

# Merge orders that got split into multiple trades
previndex = None
prevrow = None
for index, row in binanceClosedOrdersDf.iterrows():
    if previndex is not None and \
        prevrow is not None and \
        prevrow['symbol'] == row['symbol'] and \
        prevrow['side'] == row['side'] and \
        prevrow['price'] == row['price']:
            amount = prevrow['amount'] + row['amount']
            cost = prevrow['cost'] + row['cost']
            binanceClosedOrdersDf.set_value(previndex, 'amount', amount)
            binanceClosedOrdersDf.set_value(previndex, 'cost', cost)
            binanceClosedOrdersDf.drop(index, inplace=True)
            prevrow = binanceClosedOrdersDf.loc[previndex]
    else:
        previndex = index
        prevrow = row

# make buy orders cost as negative, and sold amount as negative
for index, row in binanceClosedOrdersDf.iterrows():
    if row['side'] is 'buy':
        binanceClosedOrdersDf.set_value(index, 'cost', -1 * row['cost'])
    if row['side'] is 'sell':
        binanceClosedOrdersDf.set_value(index, 'amount', -1 * row['amount'])

print(binanceClosedOrdersDf)

print(binanceClosedOrdersDf.groupby('symbol')['cost'].sum())
print(binanceClosedOrdersDf.groupby('symbol')['amount'].sum())

In [ ]:
pd.set_option('display.expand_frame_repr', False)

if binanceClosedOrdersDf.index.name is not 'symbol':
    binanceClosedOrdersDf.set_index('symbol', inplace=True)

for index in binanceClosedOrdersDf.index.unique():
    df = binanceClosedOrdersDf.loc[[index]].set_index('datetime')
    tradescount = len(df.index)
    amount = 0
    avgcost = 0
    print('---------------------')
    print(index)
    print('---------------------')
    #profitDf = pd.DataFrame()
    for ix, row in df.iterrows():
        if row['side'] is 'buy':
            avgcost = (amount*avgcost + -1*row['cost'])/ (amount + row['amount'])
        amount += row['amount']
        tradescount -= 1;
        if tradescount == 0:
            valuenow = exchanges['binance'].fetch_ticker(index)['bid']
            pandl = valuenow - row['price']
            df.set_value(ix, 'P&L', amount*pandl*BTC_USDT)
            df.set_value(ix, 'Profit', '{}%'.format(pandl/avgcost*100))
        elif row['side'] is 'sell':
            pandl = row['price'] - avgcost
            df.set_value(ix, 'P&L', '${}'.format(-1*row['amount']*pandl*BTC_USDT))
            df.set_value(ix, 'Profit', '{}%'.format(pandl/avgcost*100))
        row = df.loc[[ix]]
        print(row)
        #print('Setting avg cost: {}, amount: {}'.format(avgcost, amount))
        #profitDf.append(pd.DataFrame(row))
    #print(profitDf)

In [ ]:
import matplotlib.finance as mpf
from datetime import datetime

plt.rcParams['figure.figsize'] = (20,10)

fig = plt.figure()
ax = plt.subplot()

exchange = exchanges['binance']
ohlc = [ [x[0] / 1000] + x[1:] for x in exchange.fetch_ohlcv("XRP/BTC", '1d')[-60:]]
plt.xticks([x[0] for x in ohlc[::2]], [datetime.utcfromtimestamp(x[0]).strftime("%m/%d") for x in ohlc[::2]])

mpf.candlestick_ohlc(ax, ohlc, colorup='g', colordown='r')
ax.grid()
fig.autofmt_xdate()